In [8]:
import pandas as pd
import numpy as np

import json
from geopy.geocoders import Nominatim

ann_reports = pd.read_csv("../Charity_Data/Raw/annual-reports-15012021.csv")
pub_reg = pd.read_csv("../Charity_Data/Raw/public-register-15012021.csv").drop(['Unnamed: 11'], axis=1)

benefacts = pd.read_csv('../Charity_Data/Raw/benefacts_2021012.csv').drop_duplicates(subset="CRA", keep='first')[['CRA', 'County', 'Registered Address']]
benefacts.columns = ['Registered Charity Number', 'Benefact_county', 'Benefact_address']

In [11]:
def merge_datasets(annual_reports, public_registry):
    global all_regulator 
    all_regulator = annual_reports.merge(public_registry, on='Registered Charity Number', how='left').drop(['Registered Charity Name_y'], axis=1).rename(columns={"Registered Charity Name_x": "Registered Charity Name"}).reset_index(drop=True)
    print("MERGE DATASET SUCCESS")
    print(".............")


def include_missing_purpose(data):
    data.loc[(data['Report Activity'].str.contains("Religious activities") | data['Beneficiaries'].str.contains("Religious|Religion|Priests")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of religion"
    
    data.loc[data['Report Activity'].str.contains("Promotion of community") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of community development, including rural or urban regeneration"
    
    data.loc[data['Report Activity'].str.contains("Promotion of health") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Promotion of health, including the prevention or relief of sickness, disease or human suffering"
    
    data.loc[data['Report Activity'].str.contains("Advancement of Arts|Cultural promotion") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of the arts, culture, heritage or sciences"
    
    data.loc[data['Report Activity'].str.contains("Animal welfare") & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Prevention or relief of suffering of animals"
    
    data.loc[(data['Report Activity'].str.contains("Welfare/benevolent|Provision of accommodation/housing|Welfare of those in need|Disability support")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of community welfare including the relief of those in need by reason of youth, age, ill-health, or disability"
    
    data.loc[(data['Report Activity'].str.contains("Education|Research/evaluation|Playgroup/afterschool") | data['Beneficiaries'].str.contains("University|School|College")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of education"

    data.loc[(data['Report Activity'].str.contains("Relief of poverty|Overseas aid/famine relief")) & data['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Relief of poverty or economic hardship"

    if(len(data[data['Charitable Purpose'].isna()]) > 0):
        print("INCLUDE MISSING PURPOSE FAILED")
        print("These are the rows without a recoverable match")
        print(data[data['Charitable Purpose'].isna()])
        print(".............")
    else:
        print("INCLUDE MISSING PURPOSE SUCCESS")
        print(".............")


def condense_purpose(data):
    
    split_purposes = data['Charitable Purpose'].str.split(';', expand=True).fillna('')
    num_cols = len(split_purposes.columns)

    edu_vals = ('Advancement of education', ' Advancement of education')
    com_vals = ('Advancement of community development', ' Advancement of community development', ' Other purpose that is of benefit to the community', 'Other purpose that is of benefit to the community', 'Advancement of community welfare including the relief of those in need by reason of youth, age, ill-health, or disability', ' Advancement of community welfare including the relief of those in need by reason of youth, age, ill-health, or disability', 'Advancement of community development, including rural or urban regeneration', ' Advancement of community development, including rural or urban regeneration')
    ani_env_vals = ('Prevention or relief of suffering of animals', ' Prevention or relief of suffering of animals', 'Advancement of environmental sustainability', ' Advancement of environmental sustainability', 'Protection of the natural environment', ' Protection of the natural environment')
    arts_vals = ('Advancement of the arts, culture, heritage or sciences', ' Advancement of the arts, culture, heritage or sciences')
    prop_vals = (' Advancement of the efficient and effective use of the property of charitable organisations', 'Advancement of the efficient and effective use of the property of charitable organisations')
    conf_vals = ('Advancement of conflict resolution or reconciliation', ' Advancement of conflict resolution or reconciliation')
    rel_vals = (' Advancement of religion', 'Advancement of religion')
    soc_vals = ('Integration of those who are disadvantaged, and the promotion of their full participation, in society', ' Integration of those who are disadvantaged, and the promotion of their full participation, in society')
    civic_vals = ('Promotion of civic responsibility or voluntary work', ' Promotion of civic responsibility or voluntary work')
    health_vals = ('Promotion of health, including the prevention or relief of sickness, disease or human suffering', ' Promotion of health, including the prevention or relief of sickness, disease or human suffering')
    racism_vals = (' Promotion of religious or racial harmony and harmonious community relations', 'Promotion of religious or racial harmony and harmonious community relations')
    pov_vals = ('Relief of poverty or economic hardship', ' Relief of poverty or economic hardship')
    
    for i in range(0, num_cols):
        indexed_col = split_purposes[i]

        indexed_col[indexed_col.isin(edu_vals)] = "Education"
        indexed_col[indexed_col.isin(com_vals)] = "Community"
        indexed_col[indexed_col.isin(ani_env_vals)] = "Animals / Environment"
        indexed_col[indexed_col.isin(arts_vals)] = "Arts"
        indexed_col[indexed_col.isin(prop_vals)] = "Property"
        indexed_col[indexed_col.isin(conf_vals)] = "Conflict"
        indexed_col[indexed_col.isin(rel_vals)] = "Religion"
        indexed_col[indexed_col.isin(soc_vals)] = "Society"
        indexed_col[indexed_col.isin(civic_vals)] = "Civic Duty"
        indexed_col[indexed_col.isin(health_vals)] = "Health"    
        indexed_col[indexed_col.isin(racism_vals)] = "Racism"
        indexed_col[indexed_col.isin(pov_vals)] = "Poverty"

    data['Purpose'] = split_purposes.apply(' '.join, axis=1)

    options = 'Education|Community|Animals / Environment|Arts|Property|Conflict|Religion|Society|Civic Duty|Health|Racism|Poverty'
    num_matches = len(data['Purpose'].str.contains(options).value_counts())
    num_failed = len(data)-num_matches

    if(num_matches > 1):
        print("CONDENSE PURPOSE FAILED") 
        print("There are "+str(num_failed)+" rows without a condensable purpose, here are the long-form purposes:")
        print(data[~data['Purpose'].str.contains(options)]['Purpose'])
        print(".............")
    else:
        print("CONDENSE PURPOSE SUCCESS")
        print(".............")


def split_datasets(data):
    data['Registered Charity Number'] = data['Registered Charity Number'].apply(str)
    fin_cols = ['Registered Charity Number', 'Financial: Income from Central Government or Local Authorities',
       'Financial: Income from other public bodies',
       'Financial: Income from philantrophic organisations',
       'Financial: Income from donations',
       'Financial: Income from trading and commercial activities',
       'Financial: Income from other sources', 'Financial: Gross Income',
       'Financial: Gross Expenditure', 'Financial: Gross Income (Schools)',
       'Financial: Gross Expenditure (Schools)']
    gen_cols = ['Registered Charity Number', 'Registered Charity Name', 'Report Size',
       'Period Start Date', 'Period End Date', 'Report Activity',
       'Activity Description', 'Beneficiaries', 'Number of Volunteers',
       'Status', 'Also Known As', 'Primary Address', 'Governing Form',
       'CHY Number', 'CRO Number', 'Country Established', 'Charitable Purpose',
       'Charitable Objects', 'Purpose']

    global fin_data 
    global gen_data
    fin_data = data[fin_cols].reset_index(drop=True)
    gen_data = data[gen_cols].drop_duplicates(subset='Registered Charity Number', keep='first').reset_index(drop=True).merge(benefacts, on='Registered Charity Number', how='left') 

    new_cols = len(gen_data.columns)+len(fin_data.columns)
    old_cols = len(data.columns)+len(benefacts.columns)

    if(new_cols != old_cols):
        print("SPLIT DATASET FAILED")
        print("Mismatched Number of Columns comparing "+str(new_cols)+" in the new DFs to "+str(old_cols)+" old DFs.")
    else:
        print("SPLIT DATASET SUCCESS")
        print(".............")


def save_json_datasets():
    gen_data.to_json('../Charity_Data/Clean/general_data.json')
    fin_data.to_json('../Charity_Data/Clean/fin_data.json')
    print("SUCCESSFULLY SAVED JSON TO LOCAL DRIVE")


def find_coords(data):
    geolocator = Nominatim(user_agent="jack_charity")
    data['Longitude'] = 'Unsure'
    data['Latitude'] = 'Unsure'

    for i in range(0, len(data)):
        try:
            location = geolocator.geocode(data['Benefact_county'][i])
            data['Longitude'][i] = location.longitude
            data['Latitude'][i] = location.latitude
        else:
            pass
    
    exp_successes = len(data)-len(data[data['Benefact_county'].isna()])
    act_successes = len(data)-len(data[data['Latitude'].isna()])

    if(exp_successes != act_successes):
        print("FINDING COORDINATES FAILED")
        print(".............")
    else:
        print("FINDING COORDINATES SUCCESS")
        print(".............")

    print(str(len(data[data['Benefact_county'].isna()]))+" of the "+str(len(data))+" charities have been removed due to missing counties.")
    data = data.dropna(subset=['Benefact_county'])


SyntaxError: invalid syntax (<ipython-input-11-f3914586d73b>, line 134)

In [118]:
merge_datasets(ann_reports, pub_reg)
include_missing_purpose(all_regulator)
condense_purpose(all_regulator)
split_datasets(all_regulator)
find_coords(gen_data)

MERGE DATASET SUCCESS
.............
INCLUDE MISSING PURPOSE SUCCESS
.............
CONDENSE PURPOSE SUCCESS
.............
SPLIT DATASET SUCCESS
.............
FINDING COORDINATES FAILED
.............
518 of the 8713 charities have been removed.


In [12]:
save_json_datasets()

NameError: name 'save_json_datasets' is not defined

In [7]:
general = pd.read_csv('./Data/Clean/general_data.csv')
pd.normalize(general)

AttributeError: module 'pandas' has no attribute 'normalize'

In [10]:
json = ge
json

KeyError: 0